# Data Analysis On Flats Dataset

### Summary of Dataset:

- The dataset comprises comprehensive details of flats listed for sale in Gurgaon, featuring approximately 3,000 records and 20 attributes. These attributes encompass property characteristics, pricing, spatial dimensions, and qualitative descriptors. The fields included are:

1. `property_name`  
2. `link`  
3. `society`  
4. `price`  
5. `area`  
6. `areaWithType`  
7. `bedRoom`  
8. `bathroom`  
9. `balcony`  
10. `additionalRoom`  
11. `address`  
12. `floorNum`  
13. `facing`  
14. `agePossession`  
15. `nearbyLocations`  
16. `description`  
17. `furnishDetails`  
18. `features`  
19. `rating`  
20. `property_id`

### Dirty Data (Issues with Data):
1. area Column:
	 - The values contain unit and currency formatting (e.g., “₹ 5,000/sq.ft”) which must be parsed to extract the numeric value (e.g., 5000). This is essential for numerical analysis.

2. society Column:
	 - Inconsistencies due to case sensitivity (e.g., uppercase vs lowercase) for the same society.
	 - Some entries include extraneous information such as ratings appended during the web scraping process (likely from 99acres.com).
	 - Some entries begin with numeric prefixes or special characters. These should be cleaned using regular expressions for consistency.

3. price Column:
	 - Prices are listed using mixed units: lakhs, crores, or text (“Price on request”).
	 - Non-numeric entries (like “Price on request”) were dropped, as they cannot be used in predictive models. Only 12 such entries were found, so the impact is minimal.
	 - All remaining prices were normalized and converted into consistent numeric format (preferably in INR) for further analysis and modeling.

### Objective:
- Ensure all monetary and area-related fields are transformed into numerical (int or float) formats to enable effective statistical analysis and machine learning applications.

### Messy Data (Structural Issues):
- Unnecessary Columns:
- Columns such as link and property_id are not relevant for analysis and can be dropped.
- Feature Engineering:
- A new derived column area (in sq.ft) was computed as:
area = price / price_per_sqft. This normalized metric allows for a more meaningful comparison across listings and supports in-depth analysis.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [ ]:
df = pd.read_csv('/content/flats.csv')
df.sample(5)

,property_name,link,society,price,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating,property_id
2042,2 BHK Flat in Sector 23A Gurgaon,https://www.99acres.com/2-bhk-bedroom-apartmen...,Radhakrishna society,55 Lac,"₹ 5,000/sq.ft.",Super Built up area 1100(102.19 sq.m.),2 Bedrooms,2 Bathrooms,1 Balcony,NaN,"4719 B/104, Sector 23A Gurgaon, Gurgaon, Haryana",1st of 4 Floors,West,5 to 10 Year Old,"['HUDA Sector 23 Market', 'Palam triangle', 'P...",Hospital nearby park nearby school nearby dail...,"['1 Fan', '1 Exhaust Fan', '1 Geyser', '1 Ligh...","['Security / Fire Alarm', 'Maintenance Staff',...","['Environment5 out of 5', 'Lifestyle4 out of 5...",B70015514
1560,2 BHK Flat in Sector 85 Gurgaon,https://www.99acres.com/2-bhk-bedroom-apartmen...,SS The Leaf3.8 ★,1.2 Crore,"₹ 9,677/sq.ft.",Super Built up area 1772(164.62 sq.m.)Carpet a...,2 Bedrooms,2 Bathrooms,3+ Balconies,Store Room,"Tower 1, Sector 85 Gurgaon, Gurgaon, Haryana",9th of 26 Floors,East,1 to 5 Year Old,"['Sapphire 83 Mall', 'Dwarka Expressway', 'Cen...",Looking for a 2 bhk property for sale in gurga...,"['10 Light', '4 AC', 'No Bed', 'No Chimney', '...","['Centrally Air Conditioned', 'Water purifier'...","['Green Area4.5 out of 5', 'Construction4.5 ou...",Y69963252
776,2 BHK Flat in Sector 84 Gurgaon,https://www.99acres.com/2-bhk-bedroom-apartmen...,Alpha Corp GurgaonOne 844.0 ★,85 Lac,"₹ 6,692/sq.ft.",Super Built up area 1270(117.99 sq.m.)Carpet a...,2 Bedrooms,2 Bathrooms,3 Balconies,NaN,"Sector 84 Gurgaon, Gurgaon, Haryana",6th of 26 Floors,West,1 to 5 Year Old,"['Delhi Public School', 'Raheja Market', 'Sapp...",Available for sale 2 bhk 1270 middel floor sem...,"['2 Wardrobe', '4 Fan', '1 Exhaust Fan', '2 Ge...","['Intercom Facility', 'Lift(s)', 'Maintenance ...","['Green Area5 out of 5', 'Construction5 out of...",I69430670
1330,3 BHK Flat in Sector 95 Gurgaon,https://www.99acres.com/3-bhk-bedroom-apartmen...,ROF Ananda,37 Lac,"₹ 5,744/sq.ft.",Carpet area: 644.12 (59.84 sq.m.),3 Bedrooms,2 Bathrooms,1 Balcony,Pooja Room,"502, Sector 95 Gurgaon, Gurgaon, Haryana",6th of 14 Floors,South-East,1 to 5 Year Old,"['Metro', 'Dwarka Expressway', 'Rajeev Chowk',...",This beautiful 3 bhk flat in sector 95 gurgaon...,"['1 Water Purifier', '4 Fan', '1 Exhaust Fan',...","['Security / Fire Alarm', 'Power Back-up', 'In...","['Environment4 out of 5', 'Lifestyle4.5 out of...",X69978608
1734,3 BHK Flat in Sector 84 Gurgaon,https://www.99acres.com/3-bhk-bedroom-apartmen...,Antriksh Heights3.7 ★,92 Lac,"₹ 5,041/sq.ft.",Super Built up area 1825(169.55 sq.m.),3 Bedrooms,3 Bathrooms,3 Balconies,"Study Room,Pooja Room","Antriksh Heights, Sector 84 Gurgaon, Gurgaon, ...",2nd of 19 Floors,East,1 to 5 Year Old,"['Old Bengali Market', 'MatriKiran High School...",Sun facing lower floor unit for sale call for ...,NaN,"['Power Back-up', 'Intercom Facility', 'Lift(s...","['Green Area4.5 out of 5', 'Construction3.5 ou...",Z69973884


In [ ]:
#shape
df.shape

(3028, 20)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3028 entries, 0 to 3027
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   property_name    3028 non-null   object
 1   link             3028 non-null   object
 2   society          3027 non-null   object
 3   price            3008 non-null   object
 4   area             3015 non-null   object
 5   areaWithType     3009 non-null   object
 6   bedRoom          3009 non-null   object
 7   bathroom         3009 non-null   object
 8   balcony          3009 non-null   object
 9   additionalRoom   1695 non-null   object
 10  address          3003 non-null   object
 11  floorNum         3007 non-null   object
 12  facing           2128 non-null   object
 13  agePossession    3008 non-null   object
 14  nearbyLocations  2914 non-null   object
 15  description      3009 non-null   object
 16  furnishDetails   2204 non-null   object
 17  features         2595 non-null   

In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df.isnull().sum()

,0
property_name,0
link,0
society,1
price,20
area,13
areaWithType,19
bedRoom,19
bathroom,19
balcony,19
additionalRoom,1333


### Remove unnecessary columns

1. link - no need
2. property_id

In [ ]:
df.drop(columns=['link','property_id'],inplace=True)

In [ ]:
df.head()

,property_name,society,price,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
0,2 BHK Flat in Krishna Colony,maa bhagwati residency,45 Lac,"₹ 5,000/sq.ft.",Carpet area: 900 (83.61 sq.m.),2 Bedrooms,2 Bathrooms,1 Balcony,NaN,"Krishna Colony, Gurgaon, Haryana",4th of 4 Floors,West,1 to 5 Year Old,"['Chintapurni Mandir', 'State bank ATM', 'Pear...",So with lift.Maa bhagwati residency is one of ...,"['3 Fan', '4 Light', '1 Wardrobe', 'No AC', 'N...","['Feng Shui / Vaastu Compliant', 'Security / F...","['Environment4 out of 5', 'Safety4 out of 5', ..."
1,2 BHK Flat in Ashok Vihar,Apna Enclave,50 Lac,"₹ 7,692/sq.ft.",Carpet area: 650 (60.39 sq.m.),2 Bedrooms,2 Bathrooms,1 Balcony,NaN,"46b, Ashok Vihar, Gurgaon, Haryana",1st of 3 Floors,West,10+ Year Old,"['Chintapurni Mandir', 'Sheetla Mata Mandir', ...","Property situated on main road, railway statio...","['3 Wardrobe', '4 Fan', '1 Exhaust Fan', '1 Ge...","['Security / Fire Alarm', 'Maintenance Staff',...","['Environment4 out of 5', 'Safety4 out of 5', ..."
2,2 BHK Flat in Sohna,Tulsiani Easy in Homes,40 Lac,"₹ 6,722/sq.ft.",Carpet area: 595 (55.28 sq.m.),2 Bedrooms,2 Bathrooms,3 Balconies,NaN,"Sohna, Gurgaon, Haryana",12nd of 14 Floors,NaN,0 to 1 Year Old,"['Huda City Metro', 'Golf Course extn road', '...","This property is 15 km away from badshapur, gu...",NaN,"['Power Back-up', 'Feng Shui / Vaastu Complian...","['Environment4 out of 5', 'Safety4 out of 5', ..."
3,2 BHK Flat in Sector 61 Gurgaon,Smart World Orchard,1.47 Crore,"₹ 12,250/sq.ft.",Carpet area: 1200 (111.48 sq.m.),2 Bedrooms,2 Bathrooms,2 Balconies,Study Room,"Sector 61 Gurgaon, Gurgaon, Haryana",2nd of 4 Floors,NaN,Dec 2023,"['Sector 55-56 Metro station', 'Bestech Centra...",Near to metro station of sector 56 and opposit...,NaN,"['Security / Fire Alarm', 'Private Garden / Te...",NaN
4,2 BHK Flat in Sector 92 Gurgaon,Parkwood Westend,70 Lac,"₹ 5,204/sq.ft.",Super Built up area 1345(124.95 sq.m.),2 Bedrooms,2 Bathrooms,3 Balconies,Study Room,"Sector 92 Gurgaon, Gurgaon, Haryana",5th of 8 Floors,NaN,Under Construction,"['Yadav Clinic', 'Bangali Clinic', 'Dr. J. S. ...",We are the proud owners of this 2 bhk alongwit...,[],NaN,"['Environment5 out of 5', 'Safety3 out of 5', ..."


### Area

In [ ]:
df.rename(columns={'area':'price_per_sqft'},inplace=True)
df.head()

,property_name,society,price,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
0,2 BHK Flat in Krishna Colony,maa bhagwati residency,45 Lac,"₹ 5,000/sq.ft.",Carpet area: 900 (83.61 sq.m.),2 Bedrooms,2 Bathrooms,1 Balcony,NaN,"Krishna Colony, Gurgaon, Haryana",4th of 4 Floors,West,1 to 5 Year Old,"['Chintapurni Mandir', 'State bank ATM', 'Pear...",So with lift.Maa bhagwati residency is one of ...,"['3 Fan', '4 Light', '1 Wardrobe', 'No AC', 'N...","['Feng Shui / Vaastu Compliant', 'Security / F...","['Environment4 out of 5', 'Safety4 out of 5', ..."
1,2 BHK Flat in Ashok Vihar,Apna Enclave,50 Lac,"₹ 7,692/sq.ft.",Carpet area: 650 (60.39 sq.m.),2 Bedrooms,2 Bathrooms,1 Balcony,NaN,"46b, Ashok Vihar, Gurgaon, Haryana",1st of 3 Floors,West,10+ Year Old,"['Chintapurni Mandir', 'Sheetla Mata Mandir', ...","Property situated on main road, railway statio...","['3 Wardrobe', '4 Fan', '1 Exhaust Fan', '1 Ge...","['Security / Fire Alarm', 'Maintenance Staff',...","['Environment4 out of 5', 'Safety4 out of 5', ..."
2,2 BHK Flat in Sohna,Tulsiani Easy in Homes,40 Lac,"₹ 6,722/sq.ft.",Carpet area: 595 (55.28 sq.m.),2 Bedrooms,2 Bathrooms,3 Balconies,NaN,"Sohna, Gurgaon, Haryana",12nd of 14 Floors,NaN,0 to 1 Year Old,"['Huda City Metro', 'Golf Course extn road', '...","This property is 15 km away from badshapur, gu...",NaN,"['Power Back-up', 'Feng Shui / Vaastu Complian...","['Environment4 out of 5', 'Safety4 out of 5', ..."
3,2 BHK Flat in Sector 61 Gurgaon,Smart World Orchard,1.47 Crore,"₹ 12,250/sq.ft.",Carpet area: 1200 (111.48 sq.m.),2 Bedrooms,2 Bathrooms,2 Balconies,Study Room,"Sector 61 Gurgaon, Gurgaon, Haryana",2nd of 4 Floors,NaN,Dec 2023,"['Sector 55-56 Metro station', 'Bestech Centra...",Near to metro station of sector 56 and opposit...,NaN,"['Security / Fire Alarm', 'Private Garden / Te...",NaN
4,2 BHK Flat in Sector 92 Gurgaon,Parkwood Westend,70 Lac,"₹ 5,204/sq.ft.",Super Built up area 1345(124.95 sq.m.),2 Bedrooms,2 Bathrooms,3 Balconies,Study Room,"Sector 92 Gurgaon, Gurgaon, Haryana",5th of 8 Floors,NaN,Under Construction,"['Yadav Clinic', 'Bangali Clinic', 'Dr. J. S. ...",We are the proud owners of this 2 bhk alongwit...,[],NaN,"['Environment5 out of 5', 'Safety3 out of 5', ..."


In [ ]:
df['society'].value_counts()

,count
society,
SS The Leaf3.8 ★,73
Tulip Violet4.3 ★,40
Shapoorji Pallonji Joyville Gurugram4.0 ★,39
Signature Global Park4.0 ★,36
Shree Vardhman Victoria3.8 ★,35
Tulip Violet4.2 ★,33
Smart World Gems,32
Smart World Orchard,32
Emaar MGF Emerald Floors Premier3.8 ★,32


### Society

In [ ]:
import re
df['society'] = df[ 'society']. apply(lambda name: re.sub(r'\d+(\.\d+)?\s?★', '', str(name)).strip()).str.lower()

In [ ]:
df['society'].value_counts()

,count
society,
tulip violet4.,73
ss the leaf3.,73
shapoorji pallonji joyville gurugram4.,39
signature global park4.,36
shree vardhman victoria3.,35
smart world gems,32
emaar mgf emerald floors premier3.,32
smart world orchard,32
dlf the ultima4.,31


### Prices

In [ ]:
df[df['price'] == 'Price on Request']

,property_name,society,price,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
25,5 BHK Flat in Sector 1 Gurgaon,the new people apartments,Price on Request,NaN,Built Up area: 4500 (418.06 sq.m.),5 Bedrooms,1 Bathroom,No Balcony,NaN,"Sector 1, Sector 1 Gurgaon, Gurgaon, Haryana",Ground of 1 Floors,NaN,undefined,"['Pooja Clinic', 'Dr. Sahil Clinic', 'Prakash ...",Best in class property available at sector 1 l...,NaN,NaN,NaN
1645,4 BHK Flat in Sector 50 Gurgaon,ss hibiscus,Price on Request,NaN,Carpet area: 4800 (445.93 sq.m.),4 Bedrooms,4 Bathrooms,3 Balconies,Servant Room,"Sector 50 Gurgaon, Gurgaon, Haryana",7th of 18 Floors,East,5 to 10 Year Old,"['Baani Square', 'SS Plaza', 'Good Earth City ...",4bedroom 4800sq.Ft middle floor apartment avai...,"['4 Fan', '8 Light', 'No AC', 'No Bed', 'No Ch...","['Lift(s)', 'Water purifier', 'Swimming Pool',...","['Environment5 out of 5', 'Lifestyle5 out of 5..."
1973,1 BHK Flat in Sector 31 Gurgaon,raheja atlantis3.,Price on Request,NaN,Carpet area: 650 (60.39 sq.m.),1 Bedroom,1 Bathroom,1 Balcony,"Pooja Room,Study Room,Servant Room","Yes, Sector 31 Gurgaon, Gurgaon, Haryana",3rd of 15 Floors,East,0 to 1 Year Old,"['Huda Metro Station', 'Reliance Fresh, Indira...",It is my property and good condition and good ...,NaN,"['Feng Shui / Vaastu Compliant', 'Security / F...","['Green Area4 out of 5', 'Construction4 out of..."
2055,2 BHK Flat in Sector 79 Gurgaon,supertech araville,Price on Request,NaN,Built Up area: 1295 (120.31 sq.m.),2 Bedrooms,2 Bathrooms,No Balcony,NaN,"Sector 79, Sector 79 Gurgaon, Gurgaon, Haryana",15th of 15 Floors,NaN,undefined,NaN,"2 bath, unfurnished, 15th floor (Of 18), super...",NaN,NaN,"['Environment4 out of 5', 'Lifestyle3 out of 5..."
2358,2 BHK Flat in Sector 37D Gurgaon,the millennia 2,Price on Request,NaN,Carpet area: 552 (51.28 sq.m.),2 Bedrooms,1 Bathroom,1 Balcony,NaN,"Sector 37D Gurgaon, Gurgaon, Haryana",13rd of 13 Floors,NaN,undefined,"['Shri Balaji Hospital and Trauma Center', 'Sh...","1 bath, unfurnished, 13th floor (Of 13), at se...",NaN,NaN,"['Environment4 out of 5', 'Lifestyle4 out of 5..."
2523,2 BHK Flat in Sector 95A Gurgaon,the roselia 2,Price on Request,NaN,Carpet area: 593 (55.09 sq.m.),2 Bedrooms,2 Bathrooms,2 Balconies,NaN,"Sector 95a, Pataudi Road, Sector 95A Gurgaon, ...",14th of 14 Floors,NaN,undefined,NaN,Best in class property available at sector 95a...,NaN,NaN,"['Environment5 out of 5', 'Lifestyle4.5 out of..."
2550,3 BHK Flat in Sector-2 Sohna,supertech hilltown,Price on Request,NaN,Built Up area: 1720 (159.79 sq.m.),3 Bedrooms,3 Bathrooms,3 Balconies,NaN,"Sohna Sector 2,gurgaon, Sector-2 Sohna, Gurgao...",4th of 4 Floors,North-East,undefined,NaN,"3 bath, unfurnished, 4th floor (Of 4), overloo...",NaN,NaN,NaN
2641,2 BHK Flat in Sector 95A Gurgaon,the roselia 2,Price on Request,NaN,Carpet area: 593 (55.09 sq.m.),2 Bedrooms,2 Bathrooms,2 Balconies,NaN,"Sector 95a, Pataudi Road, Sector 95A Gurgaon, ...",14th of 14 Floors,NaN,undefined,NaN,Best in class property available at sector 95a...,NaN,NaN,"['Environment5 out of 5', 'Safety5 out of 5', ..."
2960,3 BHK Flat in Sector 104 Gurgaon,godrej summit3.,Price on Request,NaN,Super Built up area 1647(153.01 sq.m.)Carpet a...,3 Bedrooms,3 Bathrooms,3 Balconies,NaN,"1402, Sector 104 Gurgaon, Gurgaon, Haryana",14th of 17 Floors,NaN,1 to 5 Year Old,"['MG Road Metro Station', 'The Esplanade Mall'...",We are the proud owners of this 3 bhk apartmen...,[],"['Intercom Facility', 'Lift(s)', 'Maintenance ...","['Management3 out of 5', 'Green Area4.5 out of..."
2973,4 BHK Flat in Gwal Pahari,krrish provence estate,Price on Request,NaN,Built Up area: 5800 (538.84 sq.m.),4 Bedrooms,5 Bathrooms,1 Balcony,NaN,"Gwal Pahari, Gurgaon, Haryana",17th of 24 Floors,NaN,Under Construction,"['Tniy Toons School', 'Pathways School Gurgoan...",Residential apartment for sell.Located on 17th...,NaN,NaN,NaN


In [ ]:
def treat_price(x):
    if isinstance(x, float) or isinstance(x, int):
        return round(float(x), 2)

    # Ensure x is a list and has at least one element
    if isinstance(x, list) and len(x) >= 1:
        try:
            if len(x) == 2 and x[1] == 'Lac':
                return round(float(x[0]) / 100, 2)
            else:
                return round(float(x[0]), 2)
        except ValueError:
            return None  # Or np.nan if you prefer
    return None  # Or np.nan

In [ ]:
df['price'] = df['price'].str.split(' ').apply(treat_price)

In [ ]:
df.head()

,property_name,society,price,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
0,2 BHK Flat in Krishna Colony,maa bhagwati residency,0.45,"₹ 5,000/sq.ft.",Carpet area: 900 (83.61 sq.m.),2 Bedrooms,2 Bathrooms,1 Balcony,NaN,"Krishna Colony, Gurgaon, Haryana",4th of 4 Floors,West,1 to 5 Year Old,"['Chintapurni Mandir', 'State bank ATM', 'Pear...",So with lift.Maa bhagwati residency is one of ...,"['3 Fan', '4 Light', '1 Wardrobe', 'No AC', 'N...","['Feng Shui / Vaastu Compliant', 'Security / F...","['Environment4 out of 5', 'Safety4 out of 5', ..."
1,2 BHK Flat in Ashok Vihar,apna enclave,0.50,"₹ 7,692/sq.ft.",Carpet area: 650 (60.39 sq.m.),2 Bedrooms,2 Bathrooms,1 Balcony,NaN,"46b, Ashok Vihar, Gurgaon, Haryana",1st of 3 Floors,West,10+ Year Old,"['Chintapurni Mandir', 'Sheetla Mata Mandir', ...","Property situated on main road, railway statio...","['3 Wardrobe', '4 Fan', '1 Exhaust Fan', '1 Ge...","['Security / Fire Alarm', 'Maintenance Staff',...","['Environment4 out of 5', 'Safety4 out of 5', ..."
2,2 BHK Flat in Sohna,tulsiani easy in homes,0.40,"₹ 6,722/sq.ft.",Carpet area: 595 (55.28 sq.m.),2 Bedrooms,2 Bathrooms,3 Balconies,NaN,"Sohna, Gurgaon, Haryana",12nd of 14 Floors,NaN,0 to 1 Year Old,"['Huda City Metro', 'Golf Course extn road', '...","This property is 15 km away from badshapur, gu...",NaN,"['Power Back-up', 'Feng Shui / Vaastu Complian...","['Environment4 out of 5', 'Safety4 out of 5', ..."
3,2 BHK Flat in Sector 61 Gurgaon,smart world orchard,1.47,"₹ 12,250/sq.ft.",Carpet area: 1200 (111.48 sq.m.),2 Bedrooms,2 Bathrooms,2 Balconies,Study Room,"Sector 61 Gurgaon, Gurgaon, Haryana",2nd of 4 Floors,NaN,Dec 2023,"['Sector 55-56 Metro station', 'Bestech Centra...",Near to metro station of sector 56 and opposit...,NaN,"['Security / Fire Alarm', 'Private Garden / Te...",NaN
4,2 BHK Flat in Sector 92 Gurgaon,parkwood westend,0.70,"₹ 5,204/sq.ft.",Super Built up area 1345(124.95 sq.m.),2 Bedrooms,2 Bathrooms,3 Balconies,Study Room,"Sector 92 Gurgaon, Gurgaon, Haryana",5th of 8 Floors,NaN,Under Construction,"['Yadav Clinic', 'Bangali Clinic', 'Dr. J. S. ...",We are the proud owners of this 2 bhk alongwit...,[],NaN,"['Environment5 out of 5', 'Safety3 out of 5', ..."


In [ ]:
df.sample(10)

,property_name,society,price,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
781,4 BHK Flat in Sector 83 Gurgaon,vatika gurgaon 213.,1.42,"₹ 8,875/sq.ft.",Super Built up area 2390(222.04 sq.m.)Built Up...,4 Bedrooms,4 Bathrooms,3 Balconies,"Study Room,Servant Room","Block A, Sector 83 Gurgaon, Gurgaon, Haryana",7th of 14 Floors,South-West,1 to 5 Year Old,"['Huda Metro Station (Gurugram)', 'Sapphire 83...",This is a 4 bhk row flat the flat occupies a s...,NaN,"['Security / Fire Alarm', 'Feng Shui / Vaastu ...","['Green Area5 out of 5', 'Construction4.5 out ..."
2910,3 BHK Flat in Sector 61 Gurgaon,smart world orchard,1.90,"₹ 22,700/sq.ft.",Super Built up area 1398(129.88 sq.m.)Built Up...,3 Bedrooms,3 Bathrooms,3 Balconies,NaN,"Sector 61 Gurgaon, Gurgaon, Haryana",1st of 4 Floors,NaN,Within 3 months,"['Sector 55-56 Metro station', 'Bestech Centra...",The location of the property is very prime.,"['3 Light', '1 Chimney', '1 Modular Kitchen', ...","['Security / Fire Alarm', 'Lift(s)', 'Maintena...",NaN
2605,2 BHK Flat in Sector 63A Gurgaon,signature global city 63a,1.65,"₹ 15,263/sq.ft.",Super Built up area 1081(100.43 sq.m.),2 Bedrooms,2 Bathrooms,2 Balconies,NaN,"Sector 63A Gurgaon, Gurgaon, Haryana",1st of 4 Floors,NaN,Oct 2024,"['Sector 54 Chowk Metro Station', 'Paras Trini...",It's a 2bhk in a popular area (Adjoining humon...,"['1 Wardrobe', '1 Fan', '1 Light', '1 AC', 'No...",NaN,"['Environment4 out of 5', 'Safety5 out of 5', ..."
797,4 BHK Flat in Sector 110 Gurgaon,indiabulls enigma,3.45,"₹ 10,147/sq.ft.",Super Built up area 3400(315.87 sq.m.),4 Bedrooms,4 Bathrooms,3 Balconies,Servant Room,"N/a, Sector 110 Gurgaon, Gurgaon, Haryana",9th of 20 Floors,North,5 to 10 Year Old,"['Dwarka Sector 21 Metro Station', 'Gurgaon Dr...","Situated in sector 110 gurgaon, indiabulls eni...","['1 Exhaust Fan', '8 AC', '1 Modular Kitchen',...","['Security / Fire Alarm', 'Power Back-up', 'Fe...","['Environment4 out of 5', 'Lifestyle4 out of 5..."
659,3 BHK Flat in Sector 79 Gurgaon,mapsko mount ville4.,1.25,"₹ 8,389/sq.ft.",Super Built up area 1490(138.43 sq.m.)Built Up...,3 Bedrooms,3 Bathrooms,3 Balconies,Study Room,"987, Sector 79 Gurgaon, Gurgaon, Haryana",18th of 30 Floors,South,1 to 5 Year Old,"['Huda Metro Station (Gurugram)', 'Sapphire 83...",3 bhk flat available for sell in mapsko mount ...,"['3 Wardrobe', '1 Stove', '1 Chimney', '1 Modu...","['Security / Fire Alarm', 'Power Back-up', 'Fe...","['Green Area5 out of 5', 'Construction5 out of..."
1951,2 BHK Flat in Sector 91 Gurgaon,tarc maceo3.,0.96,"₹ 5,413/sq.ft.",Super Built up area 1404(130.44 sq.m.),2 Bedrooms,2 Bathrooms,3+ Balconies,NaN,"Sector 91 Gurgaon, Gurgaon, Haryana",8th of 13 Floors,East,5 to 10 Year Old,"['Manish Gallexie 91', 'Dwarka Expressway', 'R...",I want to sell my 2 bhk apartment. Its semi fu...,"['2 Wardrobe', '4 Fan', '2 Geyser', '11 Light'...","['Security / Fire Alarm', 'Intercom Facility',...","['Green Area5 out of 5', 'Construction4 out of..."
1162,3 BHK Flat in Sector 107 Gurgaon,signature global solera3.,0.51,"₹ 7,906/sq.ft.",Carpet area: 645 (59.92 sq.m.),3 Bedrooms,2 Bathrooms,2 Balconies,Others,"Sector 107 Gurgaon, Gurgaon, Haryana",9th of 14 Floors,South,1 to 5 Year Old,"['Gurgaon Dreamz Mall', 'Dwarka Expressway', ""...",Signature global solera is one of gurgaon's mo...,NaN,"['Intercom Facility', 'Lift(s)', 'Park']","['Green Area4.5 out of 5', 'Construction4.5 ou..."
1009,4 BHK Flat in Sector 77 Gurgaon,emaar mgf palm hills4.,1.65,"₹ 8,461/sq.ft.",Super Built up area 1950(181.16 sq.m.),4 Bedrooms,3 Bathrooms,3 Balconies,Servant Room,"Sector 77 Gurgaon, Gurgaon, Haryana",9th of 9 Floors,South,1 to 5 Year Old,"['Sapphire 83 Mall', 'Dwarka Expressway', 'Soh...","A beautiful location, open from two sides and ...",NaN,"['Private Garden / Terrace', 'Intercom Facilit...","['Green Area4 out of 5', 'Construction4.5 out ..."
1918,1 BHK Flat in 

# prices_per_sqft

In [ ]:
df['price_per_sqft']

,price_per_sqft
0,"₹ 5,000/sq.ft."
1,"₹ 7,692/sq.ft."
2,"₹ 6,722/sq.ft."
3,"₹ 12,250/sq.ft."
4,"₹ 5,204/sq.ft."
5,"₹ 6,269/sq.ft."
6,"₹ 13,333/sq.ft."
7,"₹ 7,860/sq.ft."
8,"₹ 8,148/sq.ft."
9,"₹ 16,885/sq.ft."


In [ ]:
df.drop(df.index[286],inplace=True)

In [ ]:
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'].str.split('/').str.get(0).str.replace('₹','').str.replace(',','').str.strip().astype('float'))

In [ ]:
df.sample(5)

,property_name,society,price,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
1854,3 BHK Flat in Manesar,the crew bos chs manesar,0.85,3695.0,Carpet area: 2300 (213.68 sq.m.),3 Bedrooms,3 Bathrooms,2 Balconies,NaN,"Manesar, Gurgaon, Haryana",2nd of 10 Floors,East,5 to 10 Year Old,"['Pooja Clinic', 'Dr. Sahil Clinic', 'Prakash ...",The crew boss sec-1 imt manesar\n \n Additiona...,NaN,"['Power Back-up', 'Intercom Facility', 'Lift(s...","['Environment4 out of 5', 'Lifestyle3.5 out of..."
1686,3 BHK Flat in Sector 90 Gurgaon,dlf new town heights 13.,1.20,6217.0,Carpet area: 1930 (179.3 sq.m.),3 Bedrooms,3 Bathrooms,3+ Balconies,"Pooja Room,Servant Room","Sector 90 Gurgaon, Gurgaon, Haryana",8th of 19 Floors,North-West,5 to 10 Year Old,"['Manish Gallexie 91', 'NH-8 IMT Manesar', 'RH...",3 bhk with servant room flat for saleMany more...,"['1 Fan', '1 Light', 'No AC', 'No Bed', 'No Ch...","['Feng Shui / Vaastu Compliant', 'Security / F...","['Green Area5 out of 5', 'Construction4 out of..."
558,3 BHK Flat in Sector 83 Gurgaon,vatika gurgaon 213.,1.25,6313.0,Super Built up area 1980(183.95 sq.m.)Built Up...,3 Bedrooms,3 Bathrooms,2 Balconies,Servant Room,"Block B, Sector 83 Gurgaon, Gurgaon, Haryana",12nd of 18 Floors,North,5 to 10 Year Old,"['Huda Metro Station (Gurugram)', 'Sapphire 83...",Check out this 3 bhk apartment for sale in vat...,NaN,"['Security / Fire Alarm', 'Feng Shui / Vaastu ...","['Green Area5 out of 5', 'Construction4.5 out ..."
946,3 BHK Flat in DLF Phase 4,dlf ridgewood estate3.,2.00,14598.0,Super Built up area 1370(127.28 sq.m.),3 Bedrooms,3 Bathrooms,1 Balcony,NaN,"DLF Phase 4, Gurgaon, Haryana",12nd of 14 Floors,North-East,10+ Year Old,"['Huda Metro Station', 'Sahara Mall', ""Oyster'...","3bhk, renovated flat is available for sale.","['4 Fan', '1 Exhaust Fan', '3 Geyser', '8 Ligh...","['Security / Fire Alarm', 'Feng Shui / Vaastu ...","['Green Area4.5 out of 5', 'Construction5 out ..."
2662,3 BHK Flat in Sector 86 Gurgaon,microtek greenburg3.,1.45,9797.0,Built Up area: 1480 (137.5 sq.m.)Carpet area: ...,3 Bedrooms,2 Bathrooms,3 Balconies,NaN,"Sector 86 Gurgaon, Gurgaon, Haryana",8th of 21 Floors,North-East,1 to 5 Year Old,"['SS Omnia Mall', 'National Highway 8', 'Dwark...","New property , just inaugurated in march 23","['1 Water Purifier', '5 Fan', '1 Fridge', '1 E...","['Feng Shui / Vaastu Compliant', 'Security / F...","['Green Area5 out of 5', 'Amenities4.5 out of ..."


In [ ]:
df['bedRoom'].value_counts()

,count
bedRoom,
3 Bedrooms,1439
2 Bedrooms,948
4 Bedrooms,481
1 Bedroom,105
5 Bedrooms,32
6 Bedrooms,3


In [ ]:
df[df['bedRoom'].isnull()]

,property_name,society,price,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
2850,2 BHK Flat in Sector 107 Gurgaon,signature global solera3.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2851,4 BHK Flat in Sector 53 Gurgaon,tulip monsella,NaN,33198.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2852,2 BHK Flat in New Palam Vihar,my home,NaN,4400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2853,2 BHK Flat in Sohna,breez global hill view,NaN,5470.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2923,3 BHK Flat in Sector 99A Gurgaon,pareena coban residences,NaN,5759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2924,1 BHK Flat in Golf Course Ext Road,ikon tower baani city centre,NaN,12437.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2925,2 BHK Flat in Sector 89 Gurgaon,greenopolis,NaN,4820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2927,4 BHK Flat in C Block Sushant Lok Phase 1,maple heights,NaN,8888.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2928,2 BHK Flat in Sector 99 Gurgaon,assotech blith,NaN,6593.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2994,4 BHK Flat in DLF Phase 4,chanderlok society,NaN,7234.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df[~df['bedRoom'].isnull()]

In [ ]:
df['bedRoom'] = df['bedRoom'].str.split(' ').str.get(0).astype('int')

In [ ]:
df.head()

,property_name,society,price,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
0,2 BHK Flat in Krishna Colony,maa bhagwati residency,0.45,5000.0,Carpet area: 900 (83.61 sq.m.),2,2 Bathrooms,1 Balcony,NaN,"Krishna Colony, Gurgaon, Haryana",4th of 4 Floors,West,1 to 5 Year Old,"['Chintapurni Mandir', 'State bank ATM', 'Pear...",So with lift.Maa bhagwati residency is one of ...,"['3 Fan', '4 Light', '1 Wardrobe', 'No AC', 'N...","['Feng Shui / Vaastu Compliant', 'Security / F...","['Environment4 out of 5', 'Safety4 out of 5', ..."
1,2 BHK Flat in Ashok Vihar,apna enclave,0.50,7692.0,Carpet area: 650 (60.39 sq.m.),2,2 Bathrooms,1 Balcony,NaN,"46b, Ashok Vihar, Gurgaon, Haryana",1st of 3 Floors,West,10+ Year Old,"['Chintapurni Mandir', 'Sheetla Mata Mandir', ...","Property situated on main road, railway statio...","['3 Wardrobe', '4 Fan', '1 Exhaust Fan', '1 Ge...","['Security / Fire Alarm', 'Maintenance Staff',...","['Environment4 out of 5', 'Safety4 out of 5', ..."
2,2 BHK Flat in Sohna,tulsiani easy in homes,0.40,6722.0,Carpet area: 595 (55.28 sq.m.),2,2 Bathrooms,3 Balconies,NaN,"Sohna, Gurgaon, Haryana",12nd of 14 Floors,NaN,0 to 1 Year Old,"['Huda City Metro', 'Golf Course extn road', '...","This property is 15 km away from badshapur, gu...",NaN,"['Power Back-up', 'Feng Shui / Vaastu Complian...","['Environment4 out of 5', 'Safety4 out of 5', ..."
3,2 BHK Flat in Sector 61 Gurgaon,smart world orchard,1.47,12250.0,Carpet area: 1200 (111.48 sq.m.),2,2 Bathrooms,2 Balconies,Study Room,"Sector 61 Gurgaon, Gurgaon, Haryana",2nd of 4 Floors,NaN,Dec 2023,"['Sector 55-56 Metro station', 'Bestech Centra...",Near to metro station of sector 56 and opposit...,NaN,"['Security / Fire Alarm', 'Private Garden / Te...",NaN
4,2 BHK Flat in Sector 92 Gurgaon,parkwood westend,0.70,5204.0,Super Built up area 1345(124.95 sq.m.),2,2 Bathrooms,3 Balconies,Study Room,"Sector 92 Gurgaon, Gurgaon, Haryana",5th of 8 Floors,NaN,Under Construction,"['Yadav Clinic', 'Bangali Clinic', 'Dr. J. S. ...",We are the proud owners of this 2 bhk alongwit...,[],NaN,"['Environment5 out of 5', 'Safety3 out of 5', ..."


In [ ]:
df['bathroom'].value_counts()

,count
bathroom,
2 Bathrooms,1047
3 Bathrooms,991
4 Bathrooms,638
5 Bathrooms,170
1 Bathroom,115
6 Bathrooms,42
7 Bathrooms,5


In [ ]:
df['bathroom'].isnull().sum()

np.int64(0)

In [ ]:
df['bathroom'] = df['bathroom'].str.split(' ').str.get(0).astype('int')

In [ ]:
df.sample(10)

,property_name,society,price,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
1479,3 BHK Flat in Sector 83 Gurgaon,vatika gurgaon 213.,1.10,6162.0,Super Built up area 1785(165.83 sq.m.)Built Up...,3,3,3+ Balconies,NaN,"Sector 83, Sector 83 Gurgaon, Gurgaon, Haryana",5th of 15 Floors,North-West,1 to 5 Year Old,"['Huda Metro Station (Gurugram)', 'Sapphire 83...",Spacious 3 bhk 1785 lower floor semifurnished ...,"['4 Fan', '1 Exhaust Fan', '16 Light', '1 Modu...","['Security / Fire Alarm', 'Feng Shui / Vaastu ...","['Green Area5 out of 5', 'Construction4.5 out ..."
1295,3 BHK Flat in Sector 83 Gurgaon,emaar palm gardens4.,1.76,9263.0,Super Built up area 1900(176.52 sq.m.)Built Up...,3,3,3 Balconies,Servant Room,"806, Sector 83 Gurgaon, Gurgaon, Haryana",5th of 16 Floors,South-East,1 to 5 Year Old,"[""V'Lante Mall"", 'NH 48 Gurugram', 'iGrow Mont...",This 3 bhk flat is located in emaar palm garde...,"['3 Wardrobe', '10 Fan', '1 Exhaust Fan', '3 G...","['Centrally Air Conditioned', 'Water purifier'...","['Green Area5 out of 5', 'Construction4 out of..."
2969,4 BHK Flat in Golf Course Ext Road,ansals api esencia,2.40,8000.0,Built Up area: 2500 (232.26 sq.m.)Carpet area:...,4,4,3+ Balconies,NaN,"D Block, Ansal Esencia, Golf Course Ext Road ,...",1st of 3 Floors,East,0 to 1 Year Old,"['Sri Radhe Krishna Temple', 'Kamal Hospital a...","418 sq yard, 2200 sq feet, east facing, 4 bhk,...","['1 Exhaust Fan', '4 Geyser', '6 Light', '6 AC...","['Security / Fire Alarm', 'Intercom Facility',...",NaN
2096,1 BHK Flat in Sector 104 Gurgaon,zara aavaas,0.20,6645.0,Built Up area: 301 (27.96 sq.m.),1,1,1 Balcony,NaN,"Sector 104, Sector 104 Gurgaon, Gurgaon, Haryana",3rd of 3 Floors,North-East,undefined,"['Ardee Mall', 'Northern Peripheral Road', 'Mp...",Best in class property available at sector 104...,NaN,NaN,"['Environment4 out of 5', 'Lifestyle4 out of 5..."
294,2 BHK Flat in Sector-11 sohna,global hill view,0.30,4599.0,Carpet area: 761 (70.7 sq.m.),2,2,2 Balconies,NaN,"Tower 1,flat 1608, Global Hill View,sohna Sect...",16th of 18 Floors,South-East,1 to 5 Year Old,NaN,This is your chance to own a 2 bhk residential...,NaN,NaN,NaN
2247,2 BHK Flat in Sector 61 Gurgaon,pioneer park3.,1.60,11594.0,Carpet area: 1380 (128.21 sq.m.),2,2,2 Balconies,Servant Room,"Sector 61 Gurgaon, Gurgaon, Haryana",21st of 23 Floors,NaN,1 to 5 Year Old,"['Sector 55-56 Rapid Metro', 'Hong Kong Bazaar...",Check out this 2 bhk flat for sale in pioneer ...,"['2 Wardrobe', 'No AC', 'No Bed', 'No Chimney'...","['Security / Fire Alarm', 'Feng Shui / Vaastu ...","['Green Area5 out of 5', 'Construction3 out of..."
295,3 BHK Flat in Sector 67 Gurgaon,ireo victory valley4.,3.49,13000.0,Carpet area: 2452 (227.8 sq.m.),3,3,3 Balconies,Servant Room,"T-B,flat Num-1901, Sector 67 Gurgaon, Gurgaon,...",19th of 40 Floors,South-East,1 to 5 Year Old,"['AIPL Joy Street Mall', 'Golf Course Extensio...",Ireo victory valley is one of gurgaon's most s...,NaN,"['Centrally Air Conditioned', 'Water purifier'...","['Connectivity5 out of 5', 'Management4.5 out ..."
1880,3 BHK Flat in Sector 82A Gurgaon,dlf the primus3.,1.85,10283.0,Super Built up area 1799(167.13 sq.m.)Carpet a...,3,3,3+ Balconies,NaN,"Sector 82A Gurgaon, Gurgaon, Haryana",2nd of 32 Floors,South,1 to 5 Year Old,"['Sapphire 83', 'NH 48', 'Cambridge Montessori...","This 3 bhk flat is located in dlf the primus, ...","['6 Fan', '1 Exhaust Fan', '4 Geyser', '15 Lig...","['Security / Fire Alarm', 'Intercom Facility',...","['Green Area5 out of 5', 'Construction4.5 out ..."
260,2 BHK Flat in Sector 90 Gurgaon,mrg world ultimus,0.50,6983.0,Carpet area: 716 (66.52 sq.m.),2,2,2 Balconies,NaN,"Sector 90 Gurgaon, Gurgaon, Haryana",1st of 14 Floors,North-East,Mar 2024,"['Huda Metro Station', 'Aanyaz Mall', 'Sector ...","Best location, beside property 200 bed running...",NaN,"['Lift(s)', 'Maintenance Staff', 'Water Storag...","['Envir

In [ ]:
df['balcony'].value_counts()

,count
balcony,
3 Balconies,977
3+ Balconies,863
2 Balconies,751
1 Balcony,318
No Balcony,99


In [ ]:
df['balcony'] = df['balcony'].str.split(' ').str.get(0).str.replace('No','0')

In [ ]:
df.sample(10)

,property_name,society,price,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
394,4 BHK Flat in Sector 15 Gurgaon,cghs rail vihar4.,1.40,10769.0,Carpet area: 1300 (120.77 sq.m.),4,3,3,NaN,"Sector 15 Gurgaon, Gurgaon, Haryana",3rd of 8 Floors,East,10+ Year Old,"['Hanuman Mandir', 'Rajiv Chowk Mosque', 'Hdfc...",Beautifully maintained and aesthetically renov...,"['1 Water Purifier', '6 Fan', '1 Exhaust Fan',...","['Security / Fire Alarm', 'Feng Shui / Vaastu ...","['Green Area4 out of 5', 'Construction4 out of..."
1770,3 BHK Flat in Sector 69 Gurgaon,tulip violet4.,1.60,10139.0,Super Built up area 1578(146.6 sq.m.),3,3,2,Pooja Room,"Sector 69 Gurgaon, Gurgaon, Haryana",Ground of 14 Floors,South-West,1 to 5 Year Old,"['Airia Mall Sector 68', 'Southern Peripheral ...",3 bhk flat on ground floor. Located in the pop...,"['1 Modular Kitchen', 'No AC', 'No Bed', 'No C...","['Feng Shui / Vaastu Compliant', 'Security / F...","['Green Area5 out of 5', 'Construction4 out of..."
2176,3 BHK Flat in Sector 51 Gurgaon,orchid island4.,1.70,10651.0,Carpet area: 1596 (148.27 sq.m.),3,4,3,"Pooja Room,Store Room","Sector 51 Gurgaon, Gurgaon, Haryana",2nd of 2 Floors,West,5 to 10 Year Old,"['Rapid Metro Sector 55-56', 'Tigra Market', '...",It's one of the huge lowrise society with mass...,"['4 Geyser', '10 Light', '4 AC', 'No Bed', 'No...","['Power Back-up', 'Private Garden / Terrace', ...","['Green Area5 out of 5', 'Construction4.5 out ..."
1972,2 BHK Flat in Sector 107 Gurgaon,signature global solera 2,0.51,9.0,Carpet area: 514396 (47788.95 sq.m.),2,2,1,NaN,"A5 301, Sector 107 Gurgaon, Gurgaon, Haryana",3rd of 13 Floors,North,0 to 1 Year Old,"['Dwarka Expressway', 'Delhi', 'IGI Airport']",It is near metro it is very beautiful,NaN,"['Security / Fire Alarm', 'Intercom Facility',...","['Environment3 out of 5', 'Lifestyle2 out of 5..."
2164,3 BHK Flat in Sector 32 Gurgaon,aqua front towers,2.50,11715.0,Built Up area: 2134 (198.26 sq.m.),3,4,3,NaN,"Sohna Sector 32, Sector 32 Gurgaon, Gurgaon, H...",1st of 1 Floors,NaN,undefined,NaN,Multistorey apartment is available for sale. I...,"['1 Light', 'No AC', 'No Bed', 'No Chimney', '...",NaN,NaN
722,3 BHK Flat in Sector 107 Gurgaon,signature global solera3.,0.52,8062.0,Carpet area: 645 (59.92 sq.m.),3,2,2,Others,"Sector 107 Gurgaon, Gurgaon, Haryana",5th of 14 Floors,East,1 to 5 Year Old,"['Gurgaon Dreamz Mall', 'Dwarka Expressway', ""...","Situated in sector 107 gurgaon, signature glob...","['3 Wardrobe', '4 Fan', '6 Light', 'No AC', 'N...","['Intercom Facility', 'Lift(s)', 'Park']","['Green Area4.5 out of 5', 'Construction4.5 ou..."
1530,4 BHK Flat in Sector 85 Gurgaon,ss,2.10,7467.0,Super Built up area 2812(261.24 sq.m.)Built Up...,4,4,3,NaN,"Sector 85 Gurgaon, Gurgaon, Haryana",2nd of 15 Floors,North-West,1 to 5 Year Old,"['Orris Community Center', 'Indian Oil', 'Essa...",Ss is one of gurgaon's most sought after desti...,"['6 AC', 'No Bed', 'No Chimney', 'No Curtains'...","['Feng Shui / Vaastu Compliant', 'Security / F...","['Environment5 out of 5', 'Lifestyle5 out of 5..."
1900,3 BHK Flat in Sector 102 Gurgaon,conscient heritage max4.,1.78,9218.0,Super Built up area 1931(179.4 sq.m.),3,4,3+,Servant Room,"N/a, Sector 102 Gurgaon, Gurgaon, Haryana",5th of 25 Floors,East,5 to 10 Year Old,"['Huda Metro Station', 'Ocus Medley Mall', 'Th...",This 3 bhk apartment is available for sale in ...,"['1 Modular Kitchen', '1 Chimney', '5 AC', 'No...","['Security / Fire Alarm', 'Power Back-up', 'Fe...","['Green Area5 out of 5', 'Construction5 out of..."
2393,3 BHK Flat in Sector 79 Gurgaon,godrej aria3.,1.70,7426.0,Super Built up area 2289(212.66 sq.m.)Carpet a...,3,4,3+,Servant Room,"Sector 79 Gurgaon, Gurgaon, Haryana",6th of 15 Floors,East,0 to 1 Year Old,"['Sapphire 83 Mall', 'Delhi Jaipur Expressway'...",This beautiful 3 bhk flat in sector 79 gurgaon...,NaN,"['Security / Fire Alarm', 'Power Back-up'

In [ ]:
df['additionalRoom'].isnull().sum()

np.int64(1314)

In [ ]:
df['additionalRoom'].fillna('not available',inplace=True)
df['additionalRoom'] = df['additionalRoom'].str.lower()

<ipython-input-73-b21d57b0b2a6>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['additionalRoom'].fillna('not available',inplace=True)


In [ ]:
df.sample(10)

,property_name,society,price,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
182,3 BHK Flat in Sector 85 Gurgaon,ss the leaf3.,2.15,8928.0,Super Built up area 2408(223.71 sq.m.),3,5,3,not available,"Sector 85 Gurgaon, Gurgaon, Haryana",2nd of 16 Floors,NaN,0 to 1 Year Old,"['Sapphire 83 Mall', 'Dwarka Expressway', 'Cen...",We are the proud owners of this 3 bhk apartmen...,[],NaN,"['Green Area4.5 out of 5', 'Amenities4 out of ..."
2881,2 BHK Flat in Sector 78 Gurgaon,raheja revanta,0.99,6100.0,Super Built up area 1621(150.6 sq.m.),2,2,3,others,"Sector 78 Gurgaon, Gurgaon, Haryana",14th of 41 Floors,East,Under Construction,"['Sapphire 83 Mall', 'NH-8', 'Bal Bharti Publi...",Residential apartment for sell.Located on 14th...,NaN,"['Lift(s)', 'Maintenance Staff', 'Swimming Poo...","['Safety4 out of 5', 'Lifestyle4 out of 5', 'E..."
1878,3 BHK Flat in Sector 108 Gurgaon,raheja vedaanta3.,0.85,4637.0,Super Built up area 1833(170.29 sq.m.)Built Up...,3,3,2,servant room,"203, Sector 108 Gurgaon, Gurgaon, Haryana",2nd of 2 Floors,North-East,1 to 5 Year Old,NaN,Check out this 3 bhk apartment for sale in rah...,"['7 Fan', '16 Light', 'No AC', 'No Bed', 'No C...","['Power Back-up', 'Feng Shui / Vaastu Complian...","['Green Area4 out of 5', 'Construction4 out of..."
2598,2 BHK Flat in Sector 99A Gurgaon,conscient habitat3.,0.35,6422.0,Carpet area: 545 (50.63 sq.m.),2,2,1,not available,"Sector 99A Gurgaon, Gurgaon, Haryana",3rd of 14 Floors,North-East,1 to 5 Year Old,NaN,Good location with proper sunlight and air,"['2 Fan', '1 Washing Machine', '6 Light', '1 A...","['Security / Fire Alarm', 'Lift(s)', 'Maintena...","['Green Area4 out of 5', 'Amenities4 out of 5'..."
1245,3 BHK Flat in Sector 37C Gurgaon,corona optus3.,1.27,7203.0,Super Built up area 1763(163.79 sq.m.)Carpet a...,3,3,3,"study room,others","Sector 37C Gurgaon, Gurgaon, Haryana",2nd of 14 Floors,South,1 to 5 Year Old,"['The Esplanade Mall', 'Gurugram Road', 'Delhi...",Corona optus is one of the most popular destin...,NaN,"['Feng Shui / Vaastu Compliant', 'Security / F...","['Green Area5 out of 5', 'Construction4 out of..."
1410,3 BHK Flat in Sector 85 Gurgaon,orris aster court premier,1.45,6839.0,Super Built up area 2120(196.95 sq.m.)Built Up...,3,4,2,servant room,"147, Sector 85 Gurgaon, Gurgaon, Haryana",3rd of 12 Floors,North,0 to 1 Year Old,"['Propose Metro Station', 'SS Omnia Mall', 'Na...",Check out this 3 bhk apartment for sale in orr...,NaN,"['Swimming Pool', 'Fitness Centre / GYM']","['Environment5 out of 5', 'Lifestyle5 out of 5..."
2292,3 BHK Flat in Sector 79 Gurgaon,bestech altura,2.69,12500.0,Super Built up area 2150(199.74 sq.m.),3,4,3+,servant room,"Tower F, 3rd Floor, Sector 79 Gurgaon, Gurgaon...",3rd of 20 Floors,NaN,0 to 1 Year Old,"['Sapphire 83 Mall', 'NH -8', 'Golf Course Ext...",Tower f \n3rd floor\nCorner property \nSwimmin...,"['4 AC', 'No Bed', 'No Chimney', 'No Curtains'...","['Feng Shui / Vaastu Compliant', 'Security / F...","['Environment4 out of 5', 'Lifestyle3 out of 5..."
2574,2 BHK Flat in Sector 89 Gurgaon,smart world gems,0.95,8903.0,Super Built up area 1067(99.13 sq.m.)Carpet ar...,2,2,2,study room,"Sector 89 Gurgaon, Gurgaon, Haryana",1st of 4 Floors,East,Dec 2023,"['Sant Soordas Sihi Metro Station', 'Airia Mal...","First floor, green facing, vast complaint",NaN,"['Security / Fire Alarm', 'Power Back-up', 'Fe...","['Environment4 out of 5', 'Lifestyle4 out of 5..."
1623,3 BHK Flat in Sector 67A Gurgaon,ireo the corridors,1.80,14658.0,Super Built up area 1921(178.47 sq.m.)Carpet a...,3,3,3,"servant room,store room","A-000, Sector 67A Gurgaon, Gurgaon, Haryana",6th of 19 Floors,North,0 to 1 Year Old,"['Huda Metro Station (Gurugram)', 'Paras Trini...",Brokers please excuse\n3 bhk +servant room + s...,"['3 Wardrobe', '1 Water Purifier', '4 Fan', '1...","['Water purifier', 'Security / Fire Alarm', 'F...","['Environment5 out of 5', 

In [ ]:
df['floorNum'].value_counts()

,count
floorNum,
2nd of 4 Floors,74
3rd of 4 Floors,71
4th of 4 Floors,63
1st of 4 Floors,61
14th of 14 Floors,50
12nd of 14 Floors,49
Ground of 14 Floors,40
10th of 14 Floors,35
7th of 14 Floors,35


In [ ]:
df['floorNum'] = df['floorNum'].str.split(' ').str.get(0).replace('Ground','0').str.replace('Basement','-1').str.replace('Lower','0').str.extract(r'(\d+)')

In [ ]:
df.sample(10)

,property_name,society,price,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
970,2 BHK Flat in Sector 106 Gurgaon,paras dews,0.88,6353.0,Super Built up area 1385(128.67 sq.m.)Built Up...,2,2,3+,not available,"Sector 106 Gurgaon, Gurgaon, Haryana",1,South,1 to 5 Year Old,"['MG Road Metro Station', 'NeoSquare Shopping ...",Located in the popular residential address of ...,"['3 Light', 'No AC', 'No Bed', 'No Chimney', '...","['Feng Shui / Vaastu Compliant', 'Security / F...",NaN
2506,3 BHK Flat in Sector 63A Gurgaon,birla navya,3.70,15416.0,Carpet area: 2400 (222.97 sq.m.),3,3,3+,not available,"Sector 63A Gurgaon, Gurgaon, Haryana",1,North,By 2024,"['Sector 54 Chowk Metro Station', 'WorldMark G...",Premium unit in birla navya drisha 1 with base...,[],"['Security / Fire Alarm', 'Power Back-up', 'Fe...","['Environment4 out of 5', 'Lifestyle4 out of 5..."
1725,4 BHK Flat in Sector 69 Gurgaon,tulip violet4.,2.00,9583.0,Super Built up area 2087(193.89 sq.m.),4,4,2,not available,"002, Sector 69 Gurgaon, Gurgaon, Haryana",0,South-East,1 to 5 Year Old,"['Airia Mall Sector 68', 'Southern Peripheral ...",Ground floor with extended balcony this 4 bhk ...,"['1 Modular Kitchen', 'No AC', 'No Bed', 'No C...","['Water purifier', 'Security / Fire Alarm', 'P...","['Green Area5 out of 5', 'Construction4 out of..."
264,4 BHK Flat in Sector 1 Imt Manesar,hsiidc sidco aravali,0.90,3648.0,Super Built up area 2467(229.19 sq.m.),4,4,3,not available,"Sector 1 Imt Manesar, Gurgaon, Haryana",7,NaN,5 to 10 Year Old,"['Pooja Clinic', 'Dr. Sahil Clinic', 'Prakash ...",We are the proud owners of this 4 bhk apartmen...,[],NaN,"['Environment5 out of 5', 'Safety4.5 out of 5'..."
1412,3 BHK Flat in Sector 82A Gurgaon,dlf the primus3.,1.85,10176.0,Super Built up area 1818(168.9 sq.m.)Carpet ar...,3,3,3,others,"555, Sector 82A Gurgaon, Gurgaon, Haryana",15,East,5 to 10 Year Old,"['Sapphire 83', 'NH 48', 'Cambridge Montessori...",Dlf the primus is one of gurgaon's most sought...,NaN,"['Intercom Facility', 'Lift(s)', 'Swimming Poo...","['Green Area5 out of 5', 'Construction4.5 out ..."
1658,3 BHK Flat in Sector 49 Gurgaon,orchid petals4.,2.38,11497.0,Super Built up area 2070(192.31 sq.m.)Built Up...,3,4,3,study room,"Opp Sapphire Mall Sohna Road Gurgaon, Sector 4...",1,West,1 to 5 Year Old,"['Sapphire Mall', 'Omaxe City Centre', 'BigBaz...","Situated in sector 49 gurgaon, orchid petals i...","['1 Fan', '1 Geyser', '1 Chimney', '1 Modular ...","['Feng Shui / Vaastu Compliant', 'Security / F...","['Green Area5 out of 5', 'Construction5 out of..."
2348,2 BHK Flat in Sector-5 Sohna,mvn athens sohna gurgaon,0.28,5833.0,Carpet area: 480 (44.59 sq.m.),2,2,1,not available,"Sector-5 Sohna, Gurgaon, Haryana",2,NaN,1 to 5 Year Old,NaN,"Near to dam dama lake, lush green and 3km away...",NaN,"['Feng Shui / Vaastu Compliant', 'Security / F...",NaN
2668,2 BHK Flat in Sector 69 Gurgaon,surendra avenue 69,0.75,6818.0,Built Up area: 1100 (102.19 sq.m.),2,2,2,not available,"Sector 69 Gurgaon, Gurgaon, Haryana",4,NaN,5 to 10 Year Old,"['Sri Radhe Krishna Temple', 'Icici bank ATM',...",Residential apartment for sell.Located in sect...,"['1 Bed', '1 Wardrobe', '1 Fan', '1 Sofa', '1 ...",NaN,"['Environment4 out of 5', 'Safety4 out of 5', ..."
1767,3 BHK Flat in Sector 69 Gurgaon,tulip violet4.,1.44,9125.0,Super Built up area 1578(146.6 sq.m.)Carpet ar...,3,3,2,pooja room,"Sector 69 Gurgaon, Gurgaon, Haryana",1,South,1 to 5 Year Old,"['Airia Mall Sector 68', 'Southern Peripheral ...",This 3 bhk apartment is available for sale in ...,"['1 Exhaust Fan', '1 Modular Kitchen', 'No AC'...","['Water purifier', 'Centrally Air Conditioned'...","['Green Area5 out of 5', 'Construction4 out of..."
129,2 BHK Flat in Sector 104 Gurgaon,godrej summit premia towers,0.90,7149.0,Built Up area: 1259 (116.96 sq.m.),2,2,0,not available,"Sector 104, Sector 104 Gurgaon, Gurgaon, Haryana",16,NaN,unde

In [ ]:
df['facing'].fillna('NA',inplace=True)

<ipython-input-78-e6ba103970bd>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['facing'].fillna('NA',inplace=True)


In [ ]:
df['facing'].value_counts("")

,count
facing,
NA,881
North-East,506
East,492
North,302
South,203
West,183
North-West,162
South-East,144
South-West,135


### added a new cols know as area which is basicaly price/price_per_sqft

In [ ]:
df.insert(loc=4,column='area',value=round((df['price']*10000000)/df['price_per_sqft']))

In [ ]:
df.insert(loc=1, column='property_type',value='flat')

In [ ]:
df.head()

,property_name,property_type,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
0,2 BHK Flat in Krishna Colony,flat,maa bhagwati residency,0.45,5000.0,900.0,Carpet area: 900 (83.61 sq.m.),2,2,1,not available,"Krishna Colony, Gurgaon, Haryana",4,West,1 to 5 Year Old,"['Chintapurni Mandir', 'State bank ATM', 'Pear...",So with lift.Maa bhagwati residency is one of ...,"['3 Fan', '4 Light', '1 Wardrobe', 'No AC', 'N...","['Feng Shui / Vaastu Compliant', 'Security / F...","['Environment4 out of 5', 'Safety4 out of 5', ..."
1,2 BHK Flat in Ashok Vihar,flat,apna enclave,0.50,7692.0,650.0,Carpet area: 650 (60.39 sq.m.),2,2,1,not available,"46b, Ashok Vihar, Gurgaon, Haryana",1,West,10+ Year Old,"['Chintapurni Mandir', 'Sheetla Mata Mandir', ...","Property situated on main road, railway statio...","['3 Wardrobe', '4 Fan', '1 Exhaust Fan', '1 Ge...","['Security / Fire Alarm', 'Maintenance Staff',...","['Environment4 out of 5', 'Safety4 out of 5', ..."
2,2 BHK Flat in Sohna,flat,tulsiani easy in homes,0.40,6722.0,595.0,Carpet area: 595 (55.28 sq.m.),2,2,3,not available,"Sohna, Gurgaon, Haryana",12,NA,0 to 1 Year Old,"['Huda City Metro', 'Golf Course extn road', '...","This property is 15 km away from badshapur, gu...",NaN,"['Power Back-up', 'Feng Shui / Vaastu Complian...","['Environment4 out of 5', 'Safety4 out of 5', ..."
3,2 BHK Flat in Sector 61 Gurgaon,flat,smart world orchard,1.47,12250.0,1200.0,Carpet area: 1200 (111.48 sq.m.),2,2,2,study room,"Sector 61 Gurgaon, Gurgaon, Haryana",2,NA,Dec 2023,"['Sector 55-56 Metro station', 'Bestech Centra...",Near to metro station of sector 56 and opposit...,NaN,"['Security / Fire Alarm', 'Private Garden / Te...",NaN
4,2 BHK Flat in Sector 92 Gurgaon,flat,parkwood westend,0.70,5204.0,1345.0,Super Built up area 1345(124.95 sq.m.),2,2,3,study room,"Sector 92 Gurgaon, Gurgaon, Haryana",5,NA,Under Construction,"['Yadav Clinic', 'Bangali Clinic', 'Dr. J. S. ...",We are the proud owners of this 2 bhk alongwit...,[],NaN,"['Environment5 out of 5', 'Safety3 out of 5', ..."


In [ ]:
df.shape

(3008, 20)

In [ ]:
df.to_csv('flats_cleaned.csv',index=False)